# 🎯 MOUAADNET-ULTRA Detection Training

**Author:** MOUAAD IDOUFKIR

---

In [ ]:
#@title 1️⃣ Setup
!nvidia-smi
!pip install -q torch torchvision tqdm pycocotools opencv-python onnx

In [ ]:
#@title 2️⃣ Config
import os

WORK = '/teamspace/studios/this_studio' if os.path.exists('/teamspace') else '/content'
REPO = f'{WORK}/MouaadNet-Ultra'
DATA = f'{WORK}/coco'
CKPT = f'{WORK}/checkpoints'

os.makedirs(DATA, exist_ok=True)
os.makedirs(CKPT, exist_ok=True)
print(f'Work: {WORK}\nData: {DATA}\nCheckpoints: {CKPT}')

In [ ]:
#@title 3️⃣ Clone Repo
if not os.path.exists(REPO):
    !git clone https://github.com/mouuuuaad/MouaadNet-Ultra.git {REPO}
else:
    !cd {REPO} && git pull
os.chdir(REPO)
print(f'✅ {os.getcwd()}')

In [ ]:
#@title 4️⃣ Download COCO (FORCE RE-DOWNLOAD if incomplete)
import os
import shutil

# Expected counts
EXPECTED_TRAIN = 118287
EXPECTED_VAL = 5000

def count_images(path):
    if not os.path.exists(path):
        return 0
    return len([f for f in os.listdir(path) if f.endswith('.jpg')])

train_count = count_images(f'{DATA}/train2017')
val_count = count_images(f'{DATA}/val2017')
anno_exists = os.path.exists(f'{DATA}/annotations/instances_train2017.json')

print(f'Current: Train={train_count}/{EXPECTED_TRAIN}, Val={val_count}/{EXPECTED_VAL}')

# Force re-download if incomplete (less than 95%)
if train_count < EXPECTED_TRAIN * 0.95:
    print('\n⚠️ Train incomplete! Deleting and re-downloading...')
    shutil.rmtree(f'{DATA}/train2017', ignore_errors=True)
    !wget -c http://images.cocodataset.org/zips/train2017.zip -O {DATA}/train2017.zip
    !cd {DATA} && unzip -q train2017.zip && rm train2017.zip
    print(f'✅ Train: {count_images(f"{DATA}/train2017")} images')

if val_count < EXPECTED_VAL * 0.95:
    print('\n⚠️ Val incomplete! Deleting and re-downloading...')
    shutil.rmtree(f'{DATA}/val2017', ignore_errors=True)
    !wget -c http://images.cocodataset.org/zips/val2017.zip -O {DATA}/val2017.zip
    !cd {DATA} && unzip -q val2017.zip && rm val2017.zip
    print(f'✅ Val: {count_images(f"{DATA}/val2017")} images')

if not anno_exists:
    print('\n⚠️ Annotations missing! Downloading...')
    shutil.rmtree(f'{DATA}/annotations', ignore_errors=True)
    !wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O {DATA}/annotations.zip
    !cd {DATA} && unzip -q annotations.zip && rm annotations.zip

# Final verification
final_train = count_images(f'{DATA}/train2017')
final_val = count_images(f'{DATA}/val2017')
print(f'\n✅ Final: Train={final_train}, Val={final_val}')

if final_train < EXPECTED_TRAIN * 0.95:
    print('❌ ERROR: Train still incomplete! Check disk space.')
else:
    print('✅ COCO Dataset Ready!')

In [ ]:
#@title 5️⃣ Train (256x256, AMP, Focal Loss, 1CycleLR)
EPOCHS = 50
BATCH_SIZE = 32

!python training/train_detection.py \
    --data {DATA} \
    --epochs {EPOCHS} \
    --batch-size {BATCH_SIZE} \
    --save-dir {CKPT} \
    --export

In [ ]:
#@title 6️⃣ Results
import torch
import matplotlib.pyplot as plt

ckpt = torch.load(f'{CKPT}/best.pt', map_location='cpu')
history = ckpt.get('history', {})

if history:
    plt.figure(figsize=(10, 4))
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'], label='Val')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.grid(True)
    plt.title(f'Best: {ckpt["best_loss"]:.4f}')
    plt.savefig(f'{CKPT}/curves.png')
    plt.show()

print(f'\nBest: {ckpt["best_loss"]:.4f}')

In [ ]:
#@title 7️⃣ Download
print('📦 Models:')
for f in os.listdir(CKPT):
    if f.endswith(('.pt', '.onnx')):
        print(f'   {f} ({os.path.getsize(f"{CKPT}/{f}")/1e6:.1f}MB)')

try:
    from google.colab import files
    files.download(f'{CKPT}/best.pt')
    files.download(f'{CKPT}/detection.onnx')
except:
    print(f'\n📁 Files at: {CKPT}')